In [2]:
# calcul matriciel
import numpy as np
# utilisation des modèles
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

2024-04-28 22:29:08.551937: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-28 22:29:08.552451: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 22:29:08.555035: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-28 22:29:08.593478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 22:29:09.665195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [3]:
class MNISTLoader:
  def __init__(self):
    self.x_train = None
    self.x_test = None
    self.x_val = None
    
    self.x_test_normalized = None
    self.x_train_normalized_80 = None
    self.x_val_normalized_20 = None
    self.y_test = None
    self.y_train_80 = None
    self.y_val_20 = None


  def load_data(self):
    (x_train, y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data()
    x_train_normalized = x_train / 255
    self.x_test_normalized = self.x_test / 255

    self.x_train, self.x_val, self.y_train_80, self.y_val_20 = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    self.x_train_normalized_80, self.x_val_normalized_20, self.y_train_80, self.y_val_20 = train_test_split(x_train_normalized, y_train, test_size=0.2, random_state=42)


loader = MNISTLoader()
loader.load_data()


In [4]:
print(loader.x_train_normalized_80.shape)
print(loader.x_train_normalized_80[0].shape)

(48000, 28, 28)
(28, 28)


# Convolutional model with raw data

In [5]:
num_classes = 10
core_size = 4
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(128, core_size, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, core_size, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(loader.x_train_normalized_80,
          loader.y_train_80,
          validation_data=(loader.x_val_normalized_20, loader.y_val_20),
          epochs=2)

model.evaluate(loader.x_test_normalized, loader.y_test)

Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:599: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8925 - loss: 0.3435 - val_accuracy: 0.9793 - val_loss: 0.0665
Epoch 2/2
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.9851 - loss: 0.0478 - val_accuracy: 0.9875 - val_loss: 0.0390
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9865 - loss: 0.0455


[0.03479255363345146, 0.9890999794006348]

# Convolutional model with handly normalized data

In [6]:
num_classes = 10
core_size = 4
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(128, core_size, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, core_size, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(loader.x_train_normalized_80,
          loader.y_train_80,
          validation_data=(loader.x_val_normalized_20, loader.y_val_20),
          epochs=2)

model.evaluate(loader.x_test_normalized, loader.y_test)

Epoch 1/2
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8958 - loss: 0.3280 - val_accuracy: 0.9738 - val_loss: 0.0873
Epoch 2/2
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9848 - loss: 0.0497 - val_accuracy: 0.9839 - val_loss: 0.0501
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9855 - loss: 0.0451


[0.03817664086818695, 0.9879999756813049]

In [7]:
import tkinter as tk
from PIL import Image, ImageDraw

# Création de la fenêtre principale
window = tk.Tk()

canvas = tk.Canvas(window, width=280, height=280, bg='white')
canvas.pack()

image = Image.new('L', (280, 280), 255)
draw = ImageDraw.Draw(image)

def clear_canvas():
    canvas.delete('all')
    global image, draw
    image = Image.new('L', (280, 280), 255)
    draw = ImageDraw.Draw(image)

clear_button = tk.Button(window, text='Clear', command=clear_canvas)
clear_button.pack()


# Fonction appelée lors du dessin
def draw_on_canvas(event):
    x, y = event.x, event.y
    canvas.create_oval(x, y, x+10, y+10, fill='black')
    draw.rectangle([x, y, x+10, y+10], fill='black')

# Fonction appelée lors de la prédiction
def predict_digit():
    # Redimensionner l'image à la taille attendue par le modèle (28x28)
    resized_image = image.resize((28, 28))

    # Prétraitement de l'image pour l'adapter au modèle (conversion en tableau numpy, normalisation, etc.)
    preprocessed_image = preprocess_image(resized_image)

    # Effectuer la prédiction avec le modèle
    predictions = model.predict(np.expand_dims([preprocessed_image], axis=-1))[0]
    prediction = np.argmax(predictions)

    # Afficher la prédiction
    prediction_label.config(text='Prediction: ' + str(prediction))

# Fonction pour prétraiter l'image avant la prédiction
def preprocess_image(image):
    # Convertir l'image en niveaux de gris
    image = image.convert('L')

    # Convertir l'image en tableau numpy
    image_array = np.array(image)
    
    # on inverse le noir et le blanc, car le dessin s'affiche mieux sur fond blanc mais notre modèle est entrainé sur fond noir
    # en gros sur l'image 255 correspond à du vide alors que pour notre modèle 255 c'est une case dessinée par l'utilisateur
    final_array = 255 - image_array
    return final_array

predict_button = tk.Button(window, text='Predict', command=predict_digit)
predict_button.pack()

prediction_label = tk.Label(window, text='Prediction: ')
prediction_label.pack()

# Capturer les événements de dessin sur la toile
canvas.bind('<B1-Motion>', draw_on_canvas)

# Lancer l'application
window.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
